In [ ]:
!pip install shap --quiet

In [1]:
from cashe import get_train_test_split, get_training_inputs, drop_nomodel_columns
from src.data_local import MergedDataPreprocessing
from src.utilities_rejection import DataFrameRejection
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplication, ModelApplicationDeployment
import pandas as pd

In [2]:
df_train, df_test = get_train_test_split()

df_train = df_train[df_train['OUTCOME'] !='APPROVED']
df_test = df_test[df_test['OUTCOME'] !='APPROVED']

In [39]:
train_rej_attr[['NOTES','NPHIES_LABEL']]

,NOTES,NPHIES_LABEL
0,Partial Approved BE-1-7,Billing
1,Partial Approved BE-1-7,Billing
4,Partial Approved BE-1-7,Billing
5,Partial Approved BE-1-7,Billing
7,Partial Approved BE-1-7,Billing
...,...,...
488826,MN-1-1Service is not clinically justified b...,Medical
488827,MN-1-1Service is not clinically justified b...,Medical
488828,BE-1-3Submission not compliant with contrac...,Billing
488831,MN-1-1Service is not clinically justified b...,Medical


In [3]:
train_rej = DataFrameRejection(df_train)
train_rej_attr = train_rej.map_rejection_label()
df_train  = train_rej.get_label_rejection()

test_rej = DataFrameRejection(df_test)
train_rej.map_rejection_label()
df_test  = train_rej.get_label_rejection()

In [4]:
df_train[['NOTES','NPHIES_LABEL']]

,NOTES,NPHIES_LABEL
398360,Rejected for the reason Sorry you are not an a...,0
398594,CV-1-1Services performed by a non-network p...,0
398595,CV-1-1Services performed by a non-network p...,0
398596,CV-1-1Services performed by a non-network p...,0
398588,CV-1-1Services performed by a non-network p...,0
...,...,...
388372,MN-1-1,1
388385,AD-1-7,0
388628,BE-1-6,0
388629,BE-1-6,0


In [5]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)
X_train = drop_nomodel_columns(X_train)
X_test = drop_nomodel_columns(X_test)
X_train

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT_SAR,LINE_ITEM_DISCOUNT,NET_VAT_AMOUNT,PATIENT_VAT_AMOUNT,VAT_PERCENTAGE,TREATMENT_TYPE_INDICATOR,SERVICE_TYPE,...,CLAIM_TYPE,TOTAL_CLAIMED_AMOUNT_SAR,TOTAL_DISCOUNT,TOTAL_DEDUCTIBLE,TOTAL_PATIENT_VATAMOUNT,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,NEW_BORN,ICD10
398360,138.00,Consultation,1.0,120.00,0.00,18.00,0.00,0.15,2,oral-health-op,...,O,120.00,0.00,0.00,0.00,4.0,oral,2134,N,K02
398594,11.38,TRANDATE INJ. 100 MG.-AMP.20ML,1.0,11.98,0.60,0.00,0.00,0.00,2,medication-codes,...,I,18251.98,2763.60,0.00,0.00,NaN,institutional,2132,N,O82
398595,15346.75,Elective classical caesarean section,1.0,15700.00,2355.00,2001.75,0.00,0.15,2,procedures,...,I,18251.98,2763.60,0.00,0.00,NaN,institutional,2132,N,O82
398596,575.00,Routine preoperative anaes assessment,1.0,500.00,0.00,75.00,0.00,0.15,2,procedures,...,I,18251.98,2763.60,0.00,0.00,NaN,institutional,2132,N,O82
398588,312.80,Cross Matching,1.0,340.00,68.00,40.80,0.00,0.15,2,laboratory,...,I,18251.98,2763.60,0.00,0.00,NaN,institutional,2132,N,O82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388372,12.39,RIAZOLE,20.0,13.41,1.01,0.00,0.00,0.00,2,medication-codes,...,O,197.57,14.83,30.00,0.00,18.0,professional,275,N,Z39.00
388385,248.63,Cryotherapy of single skin lesion,1.0,317.13,95.14,26.64,6.66,0.15,2,procedures,...,O,437.13,95.14,68.40,10.26,5.0,professional,275,N,B07
388628,9.39,TYLENOL SUPP 100 MG,10.0,10.15,0.76,0.00,0.00,0.00,2,medication-codes,...,O,171.05,3.83,33.44,0.00,22.0,professional,275,N,J03
388629,6.66,PROF 100MG-5ML SUSP,1.0,7.20,0.54,0.00,0.00,0.00,2,medication-codes,...,O,171.05,3.83,33.44,0.00,22.0,professional,275,N,J03


In [6]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:280: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resul

In [7]:
X_test_encoded

,NET_WITH_VAT,QTY,LINE_CLAIMED_AMOUNT_SAR,LINE_ITEM_DISCOUNT,NET_VAT_AMOUNT,PATIENT_VAT_AMOUNT,VAT_PERCENTAGE,TREATMENT_TYPE_INDICATOR,DURATION,QTY_STOCKED_UOM,...,CombinedText7,CombinedText8,CombinedText9,CombinedText10,CombinedText11,CombinedText12,CombinedText13,CombinedText14,CombinedText15,CombinedText16
398360,138.00,1.0,120.00,0.00,18.00,0.00,0.15,2,0.0,1,...,0,0,0,0,0,0,0,0,0,0
398594,11.38,1.0,11.98,0.60,0.00,0.00,0.00,2,1.0,1,...,6,52,0,0,0,0,0,0,0,0
398595,15346.75,1.0,15700.00,2355.00,2001.75,0.00,0.15,2,0.0,1,...,0,0,0,0,0,0,0,0,0,0
398596,575.00,1.0,500.00,0.00,75.00,0.00,0.15,2,0.0,1,...,0,0,0,0,0,0,0,0,0,0
398588,312.80,1.0,340.00,68.00,40.80,0.00,0.15,2,0.0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388372,12.39,20.0,13.41,1.01,0.00,0.00,0.00,2,5.0,1,...,0,0,0,0,0,0,0,0,0,0
388385,248.63,1.0,317.13,95.14,26.64,6.66,0.15,2,1.0,1,...,366,0,0,0,0,0,0,0,0,0
388628,9.39,10.0,10.15,0.76,0.00,0.00,0.00,2,2.0,1,...,35,6,0,0,0,0,0,0,0,0
388629,6.66,1.0,7.20,0.54,0.00,0.00,0.00,2,3.0,1,...,321,237,0,0,0,0,0,0,0,0


In [8]:
model_rejection = ModelApplication(X_train=X_train_encoded, y_train=y_train['NPHIES_LABEL'],
                                   X_test=X_test_encoded, y_test=y_test['NPHIES_LABEL'])

In [11]:
model_rejection.get_model(force_retrain=False,model_type='rejection')

XGBoost Model is loading from disk..


In [10]:
## runtime 1
model_rejection.evaluate_model()

{'XGBoost': {'Accuracy': 0.95,
  'Precision': 0.94,
  'Recall': 0.83,
  'F1 Score': 0.88}}

In [12]:
## runtime 2
model_rejection.evaluate_model()

{'XGBoost': {'Accuracy': 0.95,
  'Precision': 0.94,
  'Recall': 0.83,
  'F1 Score': 0.88}}

In [13]:
model_rejection_deployed = ModelApplicationDeployment(X_test)
model_rejection_deployed.interprete()

XGBoost Model is loading from disk..


,main_contributing_feature
0,PURCHASER_CODE
1,PURCHASER_CODE
2,PURCHASER_CODE
3,PURCHASER_CODE
4,PURCHASER_CODE
...,...
65987,PURCHASER_CODE
65988,PURCHASER_CODE
65989,PURCHASER_CODE
65990,NET_WITH_VAT


## Models Comparison

In [13]:
from src.model_train import ModelTrainer

X_train_encoded.fillna(0,inplace=True)
X_test_encoded.fillna(0,inplace=True)

model_runner = ModelTrainer(X_train=X_train_encoded,y_train=y_train['NPHIES_LABEL']
                            ,X_test=X_test_encoded,y_test=y_test['NPHIES_LABEL'])

model_runner.train_models()

[LightGBM] [Info] Number of positive: 15977, number of negative: 60890
[LightGBM] [Info] Total Bins 5673
[LightGBM] [Info] Number of data points in the train set: 76867, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.207853 -> initscore=-1.337919
[LightGBM] [Info] Start training from score -1.337919


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [14]:
result_analysis = model_runner.evaluate_models()
result_analysis

{'Decision Tree': {'Accuracy': 1.0,
  'Precision': 1.0,
  'Recall': 0.99,
  'F1 Score': 0.99},
 'LightGBM': {'Accuracy': 0.92,
  'Precision': 0.9,
  'Recall': 0.7,
  'F1 Score': 0.79},
 'SGD Classifier': {'Accuracy': 0.78,
  'Precision': 0.37,
  'Recall': 0.12,
  'F1 Score': 0.18},
 'XGBoost': {'Accuracy': 0.95,
  'Precision': 0.94,
  'Recall': 0.82,
  'F1 Score': 0.88},
 'Neural Network': {'Accuracy': 0.79,
  'Precision': 0.49,
  'Recall': 0.3,
  'F1 Score': 0.37}}

In [15]:
top_model_features = model_runner.get_decision_tree_feature_importance()

pd.DataFrame({"Column_name":X_test_encoded.columns,"Feature_importance":top_model_features}).sort_values(
    by=['Feature_importance'],ascending=False)

,Column_name,Feature_importance
29,PURCHASER_CODE,0.146008
35,CombinedText3,0.065311
33,CombinedText1,0.059334
4,NET_VAT_AMOUNT,0.055781
10,UNIT_PRICE_STOCKED_UOM,0.054122
24,TOTAL_DISCOUNT,0.047606
14,PROVIDER_DEPARTMENT_CODE,0.044674
34,CombinedText2,0.040802
23,TOTAL_CLAIMED_AMOUNT_SAR,0.040249
25,TOTAL_DEDUCTIBLE,0.036858
